<a href="https://colab.research.google.com/github/wikipathways/pfocr-pipeline/blob/main/code/bte_export.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install numpy pandas requests requests_cache rpy2

     |████████████████████████████████| 47 kB 2.8 MB/s 
     |████████████████████████████████| 127 kB 13.2 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.


In [2]:
from io import BytesIO
import json
from pathlib import Path, PurePath
from pprint import pprint
import requests
import tempfile
from zipfile import ZipFile
import io
import sys
import re
import os
import time
import datetime

import numpy as np
import pandas as pd
import requests_cache

In [3]:
requests_cache.install_cache("pfocr_cache")

In [4]:
from functools import partial

import rpy2.robjects as ro
from rpy2.ipython import html
from rpy2.robjects import default_converter, pandas2ri
from rpy2.robjects.conversion import localconverter
from rpy2.robjects.packages import importr

html.html_rdataframe = partial(html.html_rdataframe, table_class="docutils")

In [5]:
pandas2ri.activate()
base = importr("base")
readRDS = ro.r["readRDS"]
saveRDS = ro.r["saveRDS"]

In [6]:
def rds2pandas(rds_path):
    r_df = readRDS(str(rds_path))
    with localconverter(ro.default_converter + pandas2ri.converter):
        pandas_df = ro.conversion.rpy2py(r_df)
    return pandas_df

In [7]:
def pandas2rds(pandas_df, rds_path):
    with localconverter(default_converter + pandas2ri.converter) as cv:
        r_df = DataFrame(pandas_df)

    saveRDS(r_df, str(rds_path))

In [8]:
repo_dir = Path("./")

In [9]:
latest_pfocr_data_url = (
    "https://www.dropbox.com/sh/nmm0pddamgmu32m/AACCNdFXJrpQRD-EZOPe73RCa?dl=1"
)

zipfile = ZipFile(BytesIO(
    requests.get(latest_pfocr_data_url).content
))
figures_rds_path = repo_dir.joinpath("pfocr_figures_draft.rds")
zipfile.extract(str(figures_rds_path))
genes_rds_path = repo_dir.joinpath("pfocr_genes_draft.rds")
zipfile.extract(str(genes_rds_path))

'/content/pfocr_genes_draft.rds'

In [10]:
figures_df = rds2pandas(figures_rds_path).rename(
    columns={
        "figid": "pfocr_id",
        "pmcid": "pmc_id",
        "filename": "figure_filename",
        "number": "figure_number",
        "pmc_ranked_result_index": "pmc_search_index",
        "figtitle": "figure_title",
        "papertitle": "paper_title",
        "caption": "figure_caption",
        "figlink": "relative_figure_page_url",
        "reftext": "reference",
        "year": "publication_year",
    }
)
figures_df["paper_url"] = (
    "https://www.ncbi.nlm.nih.gov/pmc/articles/" + figures_df["pmc_id"]
)

figures_df["figure_page_url"] = (
    "https://www.ncbi.nlm.nih.gov"
    + figures_df["relative_figure_page_url"]
)

figures_df["figure_thumbnail_url"] = (
    "https://www.ncbi.nlm.nih.gov/pmc/articles/"
    + figures_df["pmc_id"]
    + "/bin/"
    + figures_df["figure_filename"]
)

figures_df.drop(
    columns=[
        "figure_filename",
        "relative_figure_page_url",
    ],
    inplace=True,
)

figures_df

,pfocr_id,figure_number,figure_caption,pmc_id,paper_title,reference,pmc_search_index,pathway_score,figure_title,organism,publication_year,paper_url,figure_page_url,figure_thumbnail_url
1,PMC5732092__cshperspect-CYT-028522_F2.jpg,Figure 2,Interleukin (IL)-17RA/RC signaling pathways. I...,PMC5732092,Interleukin 17 Family Cytokines: Signaling Mec...,"Leticia Monin, et al. Cold Spring Harb Perspec...",33056,0.900423,Interleukin (IL)-17RA/RC signaling pathways,Homo sapiens,2018,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC5...,https://www.ncbi.nlm.nih.govpmc/articles/PMC57...,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC5...
2,PMC5793760__cshperspect-TGF-022210_F4.jpg,Figure 4,TGF-β signaling pathways. TGF-βs signal by bin...,PMC5793760,TGF-β Signaling in Control of Cardiovascular F...,"Marie-José Goumans, et al. Cold Spring Harb Pe...",54906,0.932830,TGFB signaling pathways,Homo sapiens,2018,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC5...,https://www.ncbi.nlm.nih.govpmc/articles/PMC57...,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC5...
3,PMC5793761__cshperspect-TGF-031989_F1.jpg,Figure 1,Bone morphogenetic protein (BMP) signaling pat...,PMC5793761,Bone Morphogenetic Proteins in Vascular Homeos...,"Marie-José Goumans, et al. Cold Spring Harb Pe...",13758,0.966398,Bone morphogenetic protein (BMP) signaling pat...,Homo sapiens,2018,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC5...,https://www.ncbi.nlm.nih.govpmc/articles/PMC57...,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC5...
4,PMC5830892__cshperspect-CEL-027961_F2.jpg,Figure 2,FERM-binding partners of Crumbs3. Through its ...,PMC5830892,The Crumbs3 Polarity Protein.,Ben Margolis. Cold Spring Harb Perspect Biol. ...,38670,0.693111,FERM-binding partners of Crumbs3,Homo sapiens,2018,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC5...,https://www.ncbi.nlm.nih.govpmc/articles/PMC58...,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC5...
5,PMC5830900__cshperspect-TGF-031997_F1.jpg,Figure 1,Role of the TGF-β family in mammary gland deve...,PMC5830900,TGF-β Family Signaling in Ductal Differentiati...,"Kaoru Kahata, et al. Cold Spring Harb Perspect...",19123,0.600142,Role of the TGFB family in mammary gland devel...,Homo sapiens,2018,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC5...,https://www.ncbi.nlm.nih.govpmc/articles/PMC58...,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC5...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
79945,PMC4216988__zh20221474360006.jpg,Fig. 6,Hypothetical mechanism of activation of ENaC b...,PMC4216988,Prostasin interacts with the epithelial Na+ ch...,"Marcelo D. Carattino, et al. Am J Physiol Rena...",108774,0.143076,Hypothetical mechanism of activation of ENaC b...,Homo sapiens,2014,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC4...,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC4...,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC4...
79946,PMC2873070__nihms128887f5.jpg,Scheme 1,A schematic diagram of a proposed working mode...,PMC2873070,Apolipoprotein E mediates sulfatide depletion ...,"Hua Cheng, et al. Neurobiol Aging. ;31(7):1188...",143547,0.127176,A schematic diagram of a proposed working mode...,Homo sapiens,None,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC2...,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC2...,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC2...
79947,PMC3651446__pnas.1220523110fig06.jpg,Fig. 6,Models for nucleation of centrosomal and kinet...,PMC3651446,Aurora kinase inhibitors reveal mechanisms of ...,"Jiun-Ming Wu, et al. Proc Natl Acad Sci U S A....",159643,0.055546,Models for nucleation of centrosomal and kinet...,Homo sapiens,2013,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC3...,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC3...,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC3...
79948,PMC6770832__cancers-11-01236-g005.jpg,Figure 5,A simplified TGFβ pathway leading to p21 expre...,PMC6770832,Relevance of Non-Targeted Effects for Radiothe...,"Carmel Mothersill, et al. Cancers (Basel). 201...",618,0.140041,Simplified TGFB path

In [11]:
genes_df = rds2pandas(genes_rds_path).rename(
    columns={
        "figid": "pfocr_id",
        "pmcid": "pmc_id",
        "entrez": "ncbigene_id",
        "word": "matched_ocr_text",
        "symbol": "lexicon_term",
        "source": "lexicon_term_source",
    }
)

genes_df

,pfocr_id,pmc_id,matched_ocr_text,lexicon_term,lexicon_term_source,hgnc_symbol,ncbigene_id
0,PMC100003__mb2410470011.jpg,PMC100003,"Ga12,Gaq",G-ALPHA-q,hgnc_alias_symbol,GNAQ,2776
1,PMC100003__mb2410470011.jpg,PMC100003,Etk,ETK,hgnc_alias_symbol,BMX,660
2,PMC100003__mb2410470011.jpg,PMC100003,FAK,FAK,hgnc_alias_symbol,PTK2,5747
3,PMC100003__mb2410470011.jpg,PMC100003,AR*,AR,hgnc_symbol,AR,367
4,PMC100003__mb2410470011.jpg,PMC100003,(Src,SRC,hgnc_symbol,SRC,6714
...,...,...,...,...,...,...,...
4176411,PMC7927090__ijms-22-02194-g002.jpg,PMC8036963,MTOR,MTOR,hgnc_symbol,MTOR,2475
4176421,PMC7927090__ijms-22-02194-g002.jpg,PMC8036963,VEGFR2,VEGFR2,hgnc_alias_symbol,KDR,3791
4176431,PMC7927090__ijms-22-02194-g002.jpg,PMC8036963,TGF-B,TGFB,bioentities_symbol,TGFB1,7040
4176441,PMC7927090__ijms-22-02194-g002.jpg,PMC8036963,TGF-B,TGFB,bioentities_symbol,TGFB2,7042


In [12]:
genes_by_pfocr_id = dict()
for pfocr_id, df in genes_df[["pfocr_id", "ncbigene_id"]].groupby("pfocr_id"):
    genes_by_pfocr_id[pfocr_id] = set(df["ncbigene_id"])

In [13]:
!rm -rf pubtator-chemical pubtator-disease
!git clone https://github.com/wikipathways/pfocr-pipeline.git
!mv pfocr-pipeline/* ./
!rm -rf pfocr-pipeline

Cloning into 'pfocr-pipeline'...
remote: Enumerating objects: 163724, done.
remote: Counting objects: 100% (163724/163724), done.
remote: Compressing objects: 100% (54795/54795), done.
remote: Total 163724 (delta 108482), reused 163508 (delta 108351), pack-reused 0
Receiving objects: 100% (163724/163724), 63.78 MiB | 15.78 MiB/s, done.
Resolving deltas: 100% (108482/108482), done.
Checking out files: 100% (208471/208471), done.


In [14]:
pubtator_chemicals_by_pfocr_id = dict()
for p in repo_dir.joinpath("pubtator-chemical").glob("*.json"):
    pfocr_id = str(p.with_suffix(".jpg").name)
    with p.open("r") as f:
        denotations = json.load(f)
        if not denotations:
            #print(f"No denotations for {pfocr_id}")
            continue
        pubtator_chemicals_by_pfocr_id[pfocr_id] = denotations

data = []
for pfocr_id, denotations in pubtator_chemicals_by_pfocr_id.items():
    for denotation in denotations:
        denotation["pfocr_id"] = pfocr_id
        data.append(denotation)

pubtator_chemicals_df = pd.DataFrame.from_records(data)
pubtator_chemicals_df

,word,type,identifier,pfocr_id
0,ADCP,Chemical,MESH:C411223,PMC7363805__fimmu-11-01395-g0003.jpg
1,iodine,Chemical,MESH:D007455,PMC6880659__fendo-10-00778-g0005.jpg
2,FADO,Chemical,NaN,PMC6891791__marinedrugs-17-00639-g006.jpg
3,glycerol,Chemical,MESH:D005990,PMC6029073__12918_2018_597_Fig1_HTML.jpg
4,4-Coumaroyl-CoA,Chemical,MESH:C058644,PMC7600843__genes-11-01156-g001.jpg
...,...,...,...,...
165057,NADH,Chemical,MESH:D009243,PMC6992624__41540_2019_122_Fig3_HTML.jpg
165058,AMP,Chemical,MESH:D000249,PMC6992624__41540_2019_122_Fig3_HTML.jpg
165059,Ca2+,Chemical,MESH:D002118,PMC6989408__fcell-07-00371-g005.jpg
165060,Ca2+,Chemical,MESH:D002118,PMC6989408__fcell-07-00371-g005.jpg


In [15]:
set(pubtator_chemicals_df["type"])

{'Chemical'}

In [16]:
pubtator_diseases_by_pfocr_id = dict()
for p in repo_dir.joinpath("pubtator-disease").glob("*.json"):
    pfocr_id = str(p.with_suffix(".jpg").name)
    with p.open("r") as f:
        denotations = json.load(f)
        if not denotations:
            #print(f"No denotations for {pfocr_id}")
            continue
        pubtator_diseases_by_pfocr_id[pfocr_id] = denotations

data = []
for pfocr_id, denotations in pubtator_diseases_by_pfocr_id.items():
    for denotation in denotations:
        denotation["pfocr_id"] = pfocr_id
        data.append(denotation)

pubtator_diseases_df = pd.DataFrame.from_records(data)
pubtator_diseases_df

,word,type,identifier,pfocr_id
0,Cancer,Disease,MESH:D009369,PMC7363805__fimmu-11-01395-g0003.jpg
1,Cancer,Disease,MESH:D009369,PMC7363805__fimmu-11-01395-g0003.jpg
2,Cancer,Disease,MESH:D009369,PMC7363805__fimmu-11-01395-g0003.jpg
3,disordered,Disease,MESH:D030342,PMC6029073__12918_2018_597_Fig1_HTML.jpg
4,flavan,Disease,NaN,PMC7600843__genes-11-01156-g001.jpg
...,...,...,...,...
17944,Stemness,Disease,MESH:D020295,PMC7847682__thnov11p3489g001.jpg
17945,Primary tumor,Disease,MESH:D009369,PMC8042160__fonc-11-638873-g0001.jpg
17946,Hemolysis,Disease,MESH:D006461,PMC7531836__pcbi.1008139.g001.jpg
17947,fibrosis,Disease,MESH:D005355,PMC7933526__fphys-11-613324-g010.jpg


In [17]:
set(pubtator_diseases_df["type"])

{'Disease'}

## Export

https://github.com/wikipathways/pathway-figure-ocr/issues/16#issuecomment-1075574192
```json
{
  "_id": "PMC0000000__nihm00000000",
  "associatedWith": {
    "title": "signaling in immune response",
    "figureUrl": "https://www.ncbi.nlm.nih.gov/pmc/articles/PMC0000000/bin/nihm00000000.jpg",
    "pmc": "PMC0000000",
    "pubmed": "00000000",
    "mentions": {
        "diseases": {
              "mesh": ["D000860", "D000333"]
        },
        "chemicals": {
               "mesh": ["D0001", "D00002"]
        },
        "genes": {
               "ncbigene": ["1234", "2345"]
        }
     }
}
```

In [18]:
# https://stackoverflow.com/a/8230505/5354298
class SetEncoder(json.JSONEncoder):
    def default(self, obj):
        if isinstance(obj, set):
            return list(obj)
        return json.JSONEncoder.default(self, obj)


with open(repo_dir.joinpath("export/bte_chemicals_diseases_genes.ndjson"), "w") as f:
    for i, row in figures_df.iterrows():
        pfocr_id = row["pfocr_id"]

        chemicals = set()
        if pfocr_id in pubtator_chemicals_by_pfocr_id:
            for denotation in pubtator_chemicals_by_pfocr_id[pfocr_id]:
                if not "identifier" in denotation:
                    continue
                identifier = denotation["identifier"]
                if identifier[:4] != "MESH":
                    continue
                chemicals.add(identifier[5:])

        diseases = set()
        if pfocr_id in pubtator_diseases_by_pfocr_id:
            for denotation in pubtator_diseases_by_pfocr_id[pfocr_id]:
                if not "identifier" in denotation:
                    continue
                identifier = denotation["identifier"]
                if identifier[:4] != "MESH":
                    continue
                diseases.add(identifier[5:])

        #genes = set(genes_df[genes_df["pfocr_id"] == pfocr_id]["ncbigene_id"])
        genes = genes_by_pfocr_id.get(pfocr_id, set())

        if chemicals or diseases or genes:
            f.write(json.dumps({
                "_id": pfocr_id,
                "associatedWith": {
                    "title": row["figure_title"],
                    "figureUrl": row["figure_thumbnail_url"],
                    "pmc": row["pmc_id"],
                    #"pubmed": "",
                    "mentions": {
                        "chemicals": {
                            "mesh": chemicals
                        },
                        "diseases": {
                            "mesh": diseases
                        },
                        "genes": {
                            "ncbigene": genes
                        },
                    }
                }
            },
            cls=SetEncoder))
            
            f.write("\n")            
    
    f.write("\n")

In [19]:
!head export/bte_chemicals_diseases_genes.ndjson

{"_id": "PMC5732092__cshperspect-CYT-028522_F2.jpg", "associatedWith": {"title": "Interleukin (IL)-17RA/RC signaling pathways", "figureUrl": "https://www.ncbi.nlm.nih.gov/pmc/articles/PMC5732092/bin/cshperspect-CYT-028522_F2.jpg", "pmc": "PMC5732092", "mentions": {"chemicals": {"mesh": []}, "diseases": {"mesh": []}, "genes": {"ncbigene": ["84818", "5196", "10131", "7188", "6387", "4791", "1994", "4283", "2920", "6426", "5966", "5473", "4117", "1432", "7189", "6374", "7186", "6300", "5594", "3627", "2919", "3326", "1051", "5601", "4790", "10758", "5603", "6373", "10563", "5600", "3576", "23765", "2921", "5602", "3605", "5971", "6372", "5599", "9547", "5595", "112744", "7184", "5970"]}}}}
{"_id": "PMC5793760__cshperspect-TGF-022210_F4.jpg", "associatedWith": {"title": "TGFB signaling pathways", "figureUrl": "https://www.ncbi.nlm.nih.gov/pmc/articles/PMC5793760/bin/cshperspect-TGF-022210_F4.jpg", "pmc": "PMC5793760", "mentions": {"chemicals": {"mesh": []}, "diseases": {"mesh": []}, "genes

In [20]:
from google.colab import files

files.download("export/bte_chemicals_diseases_genes.ndjson")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>